In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier as xgb
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

In [3]:
df_train = pd.read_feather(r'tmp\train_v9_0')
df_test = pd.read_feather(r'tmp\test_v9_0')
y_df = df_train['is_promoted']
df_train = df_train.drop(columns=['is_promoted','employee_id'], axis=1)
len(df_train.columns), df_train.columns

(24,
 Index(['no_of_trainings', 'length_of_service', 'KPIs_met >80%', 'awards_won?',
        'avg_training_score', 'department_Analytics', 'department_Finance',
        'department_HR', 'department_Legal', 'department_Operations',
        'department_Procurement', 'department_R&D',
        'department_Sales & Marketing', 'department_Technology',
        'education_Bachelor's', 'education_Below Secondary',
        'education_Master's & above', 'education_Missing',
        'previous_year_rating_0.0', 'previous_year_rating_1.0',
        'previous_year_rating_2.0', 'previous_year_rating_3.0',
        'previous_year_rating_4.0', 'previous_year_rating_5.0'],
       dtype='object'))

In [4]:
X = df_train.iloc[:,:].values
y = y_df.iloc[:].values
(X.shape, y.shape)

((54808, 24), (54808,))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [5]:
X_train = X.copy()
y_train = y.copy()

In [ ]:
(np.unique(y_valid, return_counts=True), np.unique(y_train, return_counts=True))

Training Different Models and evaluating f1 score.

### XG Boost model

In [6]:
xgb_classifier = xgb(random_state=42, n_jobs=4, learning_rate=0.3, n_estimators=140, max_depth=6, min_child_weight=6, 
                 subsample=1, colsample_bytree=1, colsample_bylevel=1)
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.3, max_delta_step=0,
       max_depth=6, min_child_weight=6, missing=None, n_estimators=140,
       n_jobs=4, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [10]:
#Working XG Boost Model
#xgb_classifier = xgb(random_state=42, n_jobs=4, learning_rate=0.3, n_estimators=140, max_depth=6, min_child_weight=6, 
#                 subsample=1, colsample_bytree=1, colsample_bylevel=1)
#xgb_classifier.fit(X_train, y_train)
y_xgb_prob = xgb_classifier.predict_proba(X_valid)
#print(metrics.f1_score(y_valid, y_hat))
#print(metrics.confusion_matrix(y_valid, y_hat))

### SVC Model

In [7]:
#Working Support vector classifier.
classifier = SVC(random_state=42, C=1000, gamma=0.2, kernel='rbf', probability=True )
classifier.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
#Working Support vector classifier.
#classifier = SVC(random_state=42, C=1000, gamma=0.2, kernel='rbf' )
#classifier.fit(X_train, y_train)
y_svc_prob = classifier.predict_proba(X_valid)
#print(metrics.f1_score(y_valid, y_hat))
#print(metrics.confusion_matrix(y_valid, y_hat))

### Random Forest Model

In [13]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1, max_features=0.6, random_state=42)
m.fit(X_train, y_train)
y_hat = m.predict(X_valid)
metrics.f1_score(y_valid, y_hat)

0.5562632696390658

In [8]:
m = RandomForestClassifier(n_estimators=160, min_samples_leaf=3, n_jobs=-1, max_features=0.6, random_state=42, 
                                    criterion='entropy', max_depth=40)
m.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=40, max_features=0.6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=160, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
y_rf_prob = m.predict_proba(X_valid)

### Tpot Model

In [9]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GradientBoostingClassifier(learning_rate=0.5, max_depth=4, max_features=0.55, min_samples_leaf=18, min_samples_split=11, n_estimators=100, subsample=0.8500000000000001)),
    ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.7000000000000001, min_samples_leaf=16, min_samples_split=16, n_estimators=100)
)
exported_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5, loss='deviance', max_depth=4,
              max_features=0.55, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=No...imators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [ ]:
y_tpot_prob = exported_pipeline.predict_proba(X_valid)

### Ensembling the models

In [17]:
all_preds = np.stack((y_xgb_prob, y_rf_prob, y_tpot_prob))
all_preds.shape

(3, 16443, 2)

In [18]:
avg_preds = all_preds.mean(axis=0)
print(avg_preds.shape)
avg_preds_1 = avg_preds[:, 1]
print(avg_preds_1.shape)

(16443, 2)
(16443,)


In [19]:
n = np.array(avg_preds_1)
n[np.abs(n) < 0.4] = 0
n[np.abs(n) >= 0.4] = 1
y_hat = n

In [20]:
print (metrics.f1_score(y_valid, y_hat))
print(metrics.confusion_matrix(y_valid, y_hat))

0.5808785529715762
[[15070    40]
 [  771   562]]


# Preparing for submission.

In [10]:
df_test.drop(columns='employee_id', axis=1, inplace=True)

In [11]:
X_test = df_test.iloc[:,:].values

In [12]:
y_test_xgf = xgb_classifier.predict_proba(X_test)
y_test_svc = classifier.predict_proba(X_test)
y_test_rf = m.predict_proba(X_test)
y_test_tpot = exported_pipeline.predict_proba(X_test)

In [13]:
all_preds = np.stack((y_test_xgf, y_test_svc, y_test_rf, y_test_tpot))
#all_preds = np.stack((y_test_xgf, y_test_rf, y_test_tpot))
print(all_preds.shape)
avg_preds = all_preds.mean(axis=0)
print(avg_preds.shape)
avg_preds_1 = avg_preds[:, 1]
print(avg_preds_1.shape)

(4, 23490, 2)
(23490, 2)
(23490,)


In [14]:
n = np.array(avg_preds_1)
n[np.abs(n) < 0.4] = 0
n[np.abs(n) >= 0.4] = 1
y_test = n

In [15]:
np.unique(y_test, return_counts=True)

(array([0., 1.]), array([22702,   788], dtype=int64))

In [27]:
#y_test = m.predict(X_test)

In [16]:
load_test_csv = pd.read_csv(r'C:\Users\GJ\Desktop\Machine Learning\fastai\WNS\test.csv', low_memory=False)

In [17]:
load_test_csv.insert(13, 'is_promoted', y_test)

In [18]:
load_test_csv.insert(14, 'probabilities', avg_preds_1)

In [19]:
load_test_csv.probabilities[(load_test_csv['avg_training_score']>80) & (load_test_csv['KPIs_met >80%']==1)] = load_test_csv.probabilities[(load_test_csv['avg_training_score']>80) & (load_test_csv['KPIs_met >80%']==1)] + 0.13

C:\Users\GJ\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
load_test_csv.probabilities[(load_test_csv['avg_training_score']>80) & (load_test_csv['awards_won?']==1)] = load_test_csv.probabilities[(load_test_csv['avg_training_score']>80) & (load_test_csv['awards_won?']==1)] + 0.13

C:\Users\GJ\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
y_prob = load_test_csv.loc[:,'probabilities'].values
y_prob.shape

(23490,)

In [22]:
load_test_csv.drop(columns=['is_promoted','probabilities'], axis=0, inplace=True)

In [25]:
n = np.array(y_prob)
n[np.abs(n) < 0.4] = 0
n[np.abs(n) >= 0.4] = 1
y_test = n

In [27]:
load_test_csv.insert(2, 'is_promoted', y_test)

In [26]:
np.unique(y_test, return_counts=True)

(array([0., 1.]), array([22546,   944], dtype=int64))

In [28]:
load_test_csv.drop(columns=['department', 'region'], inplace=True)

In [29]:
load_test_csv.head()

,employee_id,is_promoted,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,0.0,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,0.0,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,0.0,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,0.0,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,0.0,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [30]:
load_test_csv.drop(columns=['education', 'gender', 'recruitment_channel','no_of_trainings','age','previous_year_rating' ], inplace=True)

In [31]:
load_test_csv.drop(columns=['length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score'], inplace=True)

In [32]:
load_test_csv.head()

,employee_id,is_promoted
0,8724,0.0
1,74430,0.0
2,72255,0.0
3,38562,0.0
4,64486,0.0


In [33]:
load_test_csv.to_csv(r'tmp/results_v9_1.csv', index=False)